In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from helperFunctions import createDataSet


In [4]:
# Load and preprocess the data using helperFunctions.py
file = "../../data/Ohio2020_processed/train/540-ws-training_processed.csv"
data_list = createDataSet(file)

# Combine data into a single DataFrame
data = pd.concat(data_list, ignore_index=True)
# Split the data into features (X) and target (y)
X = data.drop(['cbg'], axis=1)
y = data['cbg']

# Scale the data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.values.reshape(-1, 1))

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Reshape X for LSTM input (samples, time steps, features)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))


In [5]:
def create_model(input_shape):
    model = Sequential([
        LSTM(64, activation='tanh', input_shape=input_shape, return_sequences=True),
        BatchNormalization(),
        LSTM(32, activation='tanh'),
        BatchNormalization(),
        Dense(16, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001, clipnorm=1.0), loss='mse')
    return model

def train_model(X_train, y_train, X_val, y_val, epochs=20, batch_size=32):
    model = create_model((X_train.shape[1], X_train.shape[2]))
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                        validation_data=(X_val, y_val), verbose=1, callbacks=[early_stopping])
    return model, history


In [6]:
# Train the model
model, history = train_model(X_train, y_train, X_val, y_val)

Epoch 1/20
28/28 [==============================] - 9s 55ms/step - loss: 0.1188 - val_loss: 0.2626
Epoch 2/20
28/28 [==============================] - 0s 7ms/step - loss: 0.0390 - val_loss: 0.2555
Epoch 3/20
28/28 [==============================] - 0s 7ms/step - loss: 0.0313 - val_loss: 0.2524
Epoch 4/20
28/28 [==============================] - 0s 8ms/step - loss: 0.0248 - val_loss: 0.2467
Epoch 5/20
28/28 [==============================] - 0s 7ms/step - loss: 0.0228 - val_loss: 0.2413
Epoch 6/20
28/28 [==============================] - 0s 8ms/step - loss: 0.0176 - val_loss: 0.2346
Epoch 7/20
28/28 [==============================] - 0s 8ms/step - loss: 0.0202 - val_loss: 0.2294
Epoch 8/20
28/28 [==============================] - 0s 8ms/step - loss: 0.0199 - val_loss: 0.2243
Epoch 9/20
28/28 [==============================] - 0s 8ms/step - loss: 0.0170 - val_loss: 0.2130
Epoch 10/20
28/28 [==============================] - 0s 8ms/step - loss: 0.0151 - val_loss: 0.1966
Epoch 11/20
28/28 

In [8]:
# Evaluate the model on the validation set
val_loss = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")

7/7 [==============================] - 0s 3ms/step - loss: 0.0674
Validation Loss: 0.06736073642969131
